In [2]:
import duckdb
from google.cloud import bigquery

con = duckdb.connect("nps.db")

bqclient = bigquery.Client()

dataset_ref = f"nps-public-data.nps_public_data"

In [11]:
tables = bqclient.list_tables(dataset_ref)

schema = "nps_public_data"

con.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")

for table in tables:

    table_str = f"{table.project}.{table.dataset_id}.{table.table_id}"

    table = bigquery.TableReference.from_string(table_str)

    rows = bqclient.list_rows(table)

    # this is a terrible pun
    arr_rows = rows.to_arrow(create_bqstorage_client=True)

    sql_ref = f"{schema}.{table.table_id}"

    con.sql(
        f"""
        DROP TABLE IF EXISTS {sql_ref};
        CREATE TABLE {sql_ref} AS SELECT * FROM arr_rows;
        INSERT INTO {sql_ref} SELECT * FROM arr_rows;
        """
    )

    print(table_str)

con.sql("SHOW ALL tables")

40
nps-public-data.nps_public_data.activities
40
nps-public-data.nps_public_data.activities__parks
685
nps-public-data.nps_public_data.alerts
127
nps-public-data.nps_public_data.amenities
127
nps-public-data.nps_public_data.amenities__parks
127
nps-public-data.nps_public_data.amenities__parksplaces
127
nps-public-data.nps_public_data.amenities__parksvisitorcenters
127
nps-public-data.nps_public_data.amenities__parkvisitorcenters
16937
nps-public-data.nps_public_data.articles
638
nps-public-data.nps_public_data.campgrounds
10
nps-public-data.nps_public_data.events
471
nps-public-data.nps_public_data.feespasses
1281
nps-public-data.nps_public_data.lessonplans
82
nps-public-data.nps_public_data.meta
50
nps-public-data.nps_public_data.multimedia__audio
9105
nps-public-data.nps_public_data.multimedia__galleries
50000
nps-public-data.nps_public_data.multimedia__galleries__assets
10639
nps-public-data.nps_public_data.multimedia__videos
1418
nps-public-data.nps_public_data.newsreleases
505
nps

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [21]:
dbname = "nps"
con.sql(
    f"EXPORT DATABASE '{dbname}' (FORMAT PARQUET, COMPRESSION ZSTD, ROW_GROUP_SIZE 100000);"
)